In [1481]:
import pandas as pd
import numpy as np
import seaborn as sbn

# **Products**

## Carga de datos productos

In [1482]:
df_products = pd.read_csv("../dataset_amazon/amazon - amazon_product.csv")
print(f"Filas y columnas de productos dataset: {df_products.shape}")

Filas y columnas de productos dataset: (1469, 7)


# Tipos de datos y valores no nulos por columna

In [1483]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1469 entries, 0 to 1468
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1469 non-null   object
 1   product_name         1469 non-null   object
 2   category             1469 non-null   object
 3   discounted_price     1469 non-null   object
 4   actual_price         1469 non-null   object
 5   discount_percentage  1469 non-null   object
 6   about_product        1465 non-null   object
dtypes: object(7)
memory usage: 80.5+ KB


# Identificación de valores nulos

In [1484]:
print(f" Nulos en productos: \n {df_products.isnull().sum()}")

 Nulos en productos: 
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
about_product          4
dtype: int64


Para identificar qué columnas tienen nulos se usó *isnull().sum()* , para el caso de *products* se encontró que en la columna **about_product** hay 4 nulos, se tomará la decisión de imputar con *"sin info"* más adelante.

# Manejo de valores nulos

### DataFrame *products* 

```
df_products['about_product'] → selecciona la columna about_product del DataFrame df_products.
.fillna("Sin descripción") → reemplaza todos los valores nulos (NaN) de esa columna con el texto "Sin descripción".
El resultado se reasigna a la misma columna.
```

In [1485]:
df_products['about_product'] = df_products['about_product'].fillna("Sin descripción")

Comprobamos que ya no tenemos nulos

In [1486]:
print(f" Nulos en productos: \n {df_products.isnull().sum()}")

 Nulos en productos: 
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
about_product          0
dtype: int64


# Normalizamos valores en DataFrame **products**

In [1487]:
# Precios
df_products['discounted_price'] = df_products['discounted_price'].str.replace('₹','').str.replace(',','').astype(float)
df_products['actual_price'] = df_products['actual_price'].str.replace('₹','').str.replace(',','').astype(float)

# Descuento
df_products['discount_percentage'] = df_products['discount_percentage'].str.replace('%','').astype(float)


# Valores duplicados

Contamos duplicados en **products**, usando *duplicated().sum()* , nos dice cuántas columnas completas están duplicadas, es decir, todas las columnas iguales.

In [1488]:
print(df_products.duplicated().sum())

106


En el caso de **products** hay 106 filas iguales, por lo que hay que eliminar filas idénticas para cada caso.

## Eliminación de filas idénticas

In [1489]:
#products
df_products = df_products.drop_duplicates()

Revisamos **id's** , es decir en *products*, la columnas de **product_id**, ya que se espera que sean únicos.

In [1490]:
print(df_products['product_id'].duplicated().sum())

12


En el caso de *products*, aquellos que había imputado con "Sin descripción" resultaron ser repetidos, por lo que ahora sí eliminaremos esos valores de **about_product** : 

In [1491]:
# Eliminar productos cuya descripción es "Sin descripción"
df_products = df_products[df_products['about_product'] != "Sin descripción"]

In [1492]:
print(df_products['product_id'].duplicated().sum())

8


Aún se observaron 8 registros con *product_id* repetido.

In [1493]:
# IDs de productos repetidos
dup_products = df_products[df_products['product_id'].duplicated(keep=False)]
print(dup_products.sort_values(by='product_id'))


     product_id                                       product_name  \
433  B07DJLFMPS  HP 32GB Class 10 MicroSD Memory Card (U1 TF Ca...   
686  B07DJLFMPS  HP 32GB Class 10 MicroSD Memory Card (U1 TF Ca...   
15   B083342NKJ  MI Braided USB Type-C Cable for Charging Adapt...   
699  B083342NKJ  MI Braided USB Type-C Cable for Charging Adapt...   
10   B08CF3D7QR  Portronics Konnect L POR-1081 Fast Charging 3A...   
428  B08CF3D7QR  Portronics Konnect L POR-1081 Fast Charging 3A...   
2    B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
379  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
623  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
397  B09MT84WV5  Samsung EVO Plus 128GB microSDXC UHS-I U3 130M...   
641  B09MT84WV5  Samsung EVO Plus 128GB microSDXC UHS-I U3 130M...   
336  B0B5B6PQCT  boAt Wave Call Smart Watch, Smart Talk with Ad...   
587  B0B5B6PQCT  boAt Wave Call Smart Watch, Smart Talk with Ad...   
344  B0B5LVS732  Noi

### Manejo de valores duplicados en *products*

Para el caso de **products** decidí quedarme con aquellos que tenían un descuento menor, para considerar "el peor caso".
```
df_products.sort_values('discounted_price') Ordena el DataFrame de menor a mayor según discounted_price.Es decir, los precios más bajos aparecen primero.

.drop_duplicates(subset='product_id', keep='first') Busca duplicados en la columna product_id.

Para cada product_id repetido, mantiene la primera fila (la que tiene el menor discounted_price, porque ya ordenaste).
```

In [1494]:
df_products = df_products.sort_values('discounted_price').drop_duplicates(subset='product_id', keep='first')

In [1495]:
# Ver cuántos product_id están duplicados
print(f"Ver product_id duplicados: {df_products['product_id'].duplicated().sum()}")  # debe dar 0

# Ver cuántos product_id únicos hay vs total de filas
print(df_products['product_id'].nunique(), "product_id únicos")
print(len(df_products), "filas en total")

Ver product_id duplicados: 0
1351 product_id únicos
1351 filas en total


In [1496]:
df_products.shape

(1351, 7)

# **Reviews**

## Carga de datos reviews

In [1497]:
df_reviews = pd.read_csv("../dataset_amazon/amazon - amazon_review.csv")
print(f"Filas y columnas de reviews dataset: {df_reviews.shape}")

Filas y columnas de reviews dataset: (1465, 10)


# Tipos de datos y valores no nulos por columna

In [1498]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         1465 non-null   object
 1   user_name       1465 non-null   object
 2   review_id       1465 non-null   object
 3   review_title    1465 non-null   object
 4   review_content  1465 non-null   object
 5   img_link        999 non-null    object
 6   product_link    999 non-null    object
 7   product_id      1465 non-null   object
 8   rating          1465 non-null   object
 9   rating_count    1463 non-null   object
dtypes: object(10)
memory usage: 114.6+ KB


De igual manera para *reviews* se identificaron valores nulos, en este caso **img_links** y **product_link** tienen 466 valores nulos, sin embargo, considerando las preguntas a responder en este análisis, esas columnas no nos aportan información relevante, por lo que se sugiere eliminar esas columnas. 

Por otro lado para **rating_count** hay 2 valores nulos, en este caso, se plantea imputarlos con 0.

In [1499]:
print(f" Nulos en reviews: \n  {df_reviews.isnull().sum()}")

 Nulos en reviews: 
  user_id             0
user_name           0
review_id           0
review_title        0
review_content      0
img_link          466
product_link      466
product_id          0
rating              0
rating_count        2
dtype: int64


### DataFrame *reviews* 

```
.drop(columns=[...]) → elimina columnas completas del DataFrame. Aquí borramos img_link y product_link.
Se reasigna a df_reviews para guardar el DataFrame ya sin esas columnas.
```
Imputamos nulos en rating_count con ceros.

In [1500]:
df_reviews = df_reviews.drop(columns=['img_link', 'product_link'])

In [1501]:
# Imputar con 0 si no hay valor, indicando que no se registró número de reseñas
df_reviews['rating_count'] = df_reviews['rating_count'].fillna(0)


In [1502]:

print(f" Nulos en reviews: \n  {df_reviews.isnull().sum()}")

 Nulos en reviews: 
  user_id           0
user_name         0
review_id         0
review_title      0
review_content    0
product_id        0
rating            0
rating_count      0
dtype: int64


De igual manera contamos duplicados en *reviews* 

In [1503]:
print(df_reviews.duplicated().sum())

70


 En **reviews** hubo 70 filas iguales, por lo que hay que eliminar filas idénticas.

# Eliminación de filas idénticas

In [1504]:
#reviews
df_reviews = df_reviews.drop_duplicates()


## Revisión de IDs en reviews

Revisamos los identificadores en el dataset de reseñas. En particular:

- **product_id** debería ser único para cada producto.

- **review_id** identifica cada reseña individual, pero para nuestro análisis agregado por producto, nos interesa una fila por producto, consolidando la información de todas sus reseñas.

> Nota: Al mantener una fila por **product_id**, perdemos la granularidad de las reseñas individuales, pero simplifica el análisis de métricas por producto como promedio de rating, precio, descuento, etc.

In [1505]:
# Revisar cuántos product_id duplicados hay
print(f"Duplicados en product_id (dentro de reviews) antes de limpiar: {df_reviews['product_id'].duplicated().sum()}")  

Duplicados en product_id (dentro de reviews) antes de limpiar: 44


Se observaron 44 duplicados de **product_id** en *reviews*.

## Manejo de duplicados por producto en *reviews*

In [1506]:
df_reviews = df_reviews.drop_duplicates(subset=['product_id'])

In [1507]:
# Ver cuántos product_id están duplicados
print(f"Ver product_id duplicados: {df_reviews['product_id'].duplicated().sum()}")  # debe dar 0 

# Ver cuántos product_id únicos hay vs total de filas 
print(df_reviews['product_id'].nunique(), "product_id únicos") 
print(len(df_reviews), "filas en total")


Ver product_id duplicados: 0
1351 product_id únicos
1351 filas en total


## Revisamos tipos de datos

In [1508]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1351 entries, 0 to 1464
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         1351 non-null   object
 1   user_name       1351 non-null   object
 2   review_id       1351 non-null   object
 3   review_title    1351 non-null   object
 4   review_content  1351 non-null   object
 5   product_id      1351 non-null   object
 6   rating          1351 non-null   object
 7   rating_count    1351 non-null   object
dtypes: object(8)
memory usage: 95.0+ KB


### Castear **rating** y **rating_count**

In [1509]:
# Quitar comas y convertir rating_count a int
df_reviews['rating_count'] = df_reviews['rating_count'].str.replace(',', '')
df_reviews['rating_count'] = pd.to_numeric(df_reviews['rating_count'], errors='coerce')

# Convertir rating a float
df_reviews['rating'] = pd.to_numeric(df_reviews['rating'], errors='coerce')


# Imputar valores faltantes
df_reviews['rating_count'] = df_reviews['rating_count'].fillna(0).astype(int)
df_reviews['rating'] = df_reviews['rating'].fillna(0.0)

In [1510]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1351 entries, 0 to 1464
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         1351 non-null   object 
 1   user_name       1351 non-null   object 
 2   review_id       1351 non-null   object 
 3   review_title    1351 non-null   object 
 4   review_content  1351 non-null   object 
 5   product_id      1351 non-null   object 
 6   rating          1351 non-null   float64
 7   rating_count    1351 non-null   int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 95.0+ KB


In [1511]:
df_reviews.shape

(1351, 8)